In [1]:
import json
from tqdm import tqdm
from transformers import BertTokenizerFast

In [2]:
tokenizer_fast = BertTokenizerFast.from_pretrained('save_tokenizer/')
tokenizer_fast

PreTrainedTokenizerFast(name_or_path='save_tokenizer/', vocab_size=21128, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [3]:
with open('datasets/train.conll_convert.conll', 'r', encoding='utf-8') as f:
    relation_sentences = f.read().split('\n\n')
    train_data = list()

    n = 0

    for relation_sentence in tqdm(relation_sentences[:-1]):
        # example:{"[2, 4, '阴性表现', '对称']\t[0, 2, '器官组织', '胸廓']": '属性', "[7, 9, '阴性表现', '居中']\t[5, 7, '器官组织', '气管']": '属性', xxxxxx}
        relation_dict = dict()
        for sentence in relation_sentence.split('\n'):
            try:
                ners_sentence = json.loads(sentence)
            except Exception as e:
                # example:[82, 84, '器官组织', '心影']	[84, 86, '异常现象', '改变']	属性
                # example:[35, 37, '器官组织', '心影']	[]	属性
                sub, obj, relation_type = sentence.split('\t')
                if (sub != '[]') and (obj != '[]'):
                    relation_dict[f'{sub}\t{obj}'] = relation_type

        offset_mapping_output = tokenizer_fast(ners_sentence['sent'], return_offsets_mapping=True)["offset_mapping"]
        # 处理原句空格
        offset_mapping = []
        for i, (start, end) in enumerate(offset_mapping_output):
            if (end > 0) and (i >= 2):
                start -= (i - 1)
                end -= (i - 1)
            offset_mapping.append((start, end))

        start_mapping = {j[0]: i for i, j in enumerate(offset_mapping) if j != (0, 0)}
        # j[1] - 1表示该token结尾字符的位置
        end_mapping = {j[1] - 1: i for i, j in enumerate(offset_mapping) if j != (0, 0)}

        # example:{"[0, 2, '器官组织', '胸廓']": [1, 2], "[2, 4, '阴性表现', '对称']": [3, 4], xxxxxx}
        ner_idx_dict = dict()
        for ent in ners_sentence['ners']:
            start_idx, end_idx, entity_type, entity_text = ent[0], ent[1] - 1, ent[2], ent[3]
            if start_idx in start_mapping and end_idx in end_mapping:
                start_span = start_mapping[start_idx]
                end_span = end_mapping[end_idx]
            ner_idx_dict[str(ent)] = [start_span, end_span]

        # example:{"[0, 2, '器官组织', '胸廓']": [94, 95], "[2, 4, '阴性表现', '对称']": [96, 97], xxxxxx}
        ner_token_idx_dict = dict()
        # [CLS] 胸 廓 对 称 ， 气 管 居 中 。 所 见 骨 骼 骨 质 结 构 完 整 。 双 肺 纹 理 清 晰 。 两 肺 门 影 不 大 。 心 影 横 径 增 大 ， 左 心 缘 饱 满 。 两 侧 膈 面 光 整 ， 两 侧 肋 膈 角 锐 利 。 1 . 两 肺 未 见 明 显 活 动 性 病 变 ， 随 诊 。 2 . 心 影 改 变 请 结 合 临 床 。 [SEP] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阳性表现] [/阳性表现] [器官组织] [/器官组织] [阳性表现] [/阳性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [否定描述] [/否定描述] [修饰描述] [/修饰描述] [修饰描述] [/修饰描述] [异常现象] [/异常现象] [器官组织] [/器官组织] [异常现象] [/异常现象]
        sentence_text = '[CLS] ' + ners_sentence['sent'] + ' [SEP]'
        position_ids = [i for i in range(len(offset_mapping))]
        for ner in ners_sentence['ners']:
            sentence_text += f' [{ner[2]}] [/{ner[2]}]'  # 加入实体对应的类型
            ner_token_idx_dict[str(ner)] = [len(offset_mapping) + i for i in
                                            range(2)]  # 实体对应的类型('[修饰描述]', '[/修饰描述]')在sentence_text中的位置
            position_ids.extend(ner_idx_dict[str(ner)])  # 实体对应的类型('[修饰描述]', '[/修饰描述]')与实体共享位置编码信息

        relation_label, relation_idx = [], []
        for sub in ners_sentence['ners']:  # sub:[0, 2, '器官组织', '胸廓']
            for obj in ners_sentence['ners']:
                if str(sub) == str(obj):
                    continue
                relation_label.append(relation_dict.get(f'{str(sub)}\t{str(obj)}', '无'))  # 有关系则为"属性",否则为"无"
                # 所有可能有关系的实体对象的类型组合在sentence_text中的token位置,example:[[94, 95, 96, 97], [94, 95, 98, 99], xxxxxx]
                relation_idx.append(ner_token_idx_dict[str(sub)] + ner_token_idx_dict[str(obj)])
        train_data.append({'sentence_text': sentence_text,
                           'position_ids': position_ids,
                           'relation_label': relation_label,
                           'relation_idx': relation_idx})
        if n == 0:
            print(sentence_text, end='\n\n')
            print(position_ids, end='\n\n')
            # print(relation, end='\n\n')
            print(relation_label, end='\n\n')
            print(relation_idx, end='\n\n')
            n += 1

  1%|          | 37/3953 [00:00<00:21, 185.34it/s]

[CLS] 胸 廓 对 称 ， 气 管 居 中 。 所 见 骨 骼 骨 质 结 构 完 整 。 双 肺 纹 理 清 晰 。 两 肺 门 影 不 大 。 心 影 横 径 增 大 ， 左 心 缘 饱 满 。 两 侧 膈 面 光 整 ， 两 侧 肋 膈 角 锐 利 。 1 . 两 肺 未 见 明 显 活 动 性 病 变 ， 随 诊 。 2 . 心 影 改 变 请 结 合 临 床 。 [SEP] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阳性表现] [/阳性表现] [器官组织] [/器官组织] [阳性表现] [/阳性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [否定描述] [/否定描述] [修饰描述] [/修饰描述] [修饰描述] [/修饰描述] [异常现象] [/异常现象] [器官组织] [/器官组织] [异常现象] [/异常现象]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 1, 2, 3, 4, 6, 7, 8, 9, 13, 

100%|██████████| 3953/3953 [00:24<00:00, 158.53it/s]


In [4]:
with open('datasets/train_rel.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)